In [2]:
import os
os.chdir(os.path.abspath(".."))

In [3]:
import numpy as np
import pandas as pd

from dataio import load_metadataset
from sklearn.feature_selection import SelectFpr

In [30]:
meta_X["mean_sample_entropy"][8]

inf

In [16]:
meta_X, meta_y = load_metadataset("comprehensive", problem_type="regression")

selector = SelectFpr()
selector.fit(meta_X, meta_y)
selector.get_feature_names_out()

ValueError: Input X contains NaN.
SelectFpr does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values